In [1]:
import pydeequ

from pyspark.sql import SparkSession, Row


spark = (SparkSession
    .builder
    .config("spark.driver.extraClassPath", "/Users/kimminho/spark/spark-3.1.2-bin-hadoop3.2/jars")
    .config("spark.jars.packages", pydeequ.deequ_maven_coord)
    .config("spark.jars.excludes", pydeequ.f2j_maven_coord)
    .getOrCreate())
spark

Please set env variable SPARK_VERSION


In [2]:
df = spark.read.csv('./ST_DMT_CLIENT_KEY_POINT_MGT_IDX.csv')

df.show()

+----+---+---+----+---+---+----+--------+---+--------+
| _c0|_c1|_c2| _c3|_c4|_c5| _c6|     _c7|_c8|     _c9|
+----+---+---+----+---+---+----+--------+---+--------+
|2019|  1| 02|null|  0|  0| 500| 1000000|250|  900000|
|2010|  4| 12| asd|  1|  1| 100| 2000000|450| 1900000|
|2015|  4| 12|null|  1|  0| 300| 3000000|350| 3900000|
|1994|  2| 04|null|  2|  0|  50|  500000|250| -900000|
|1996|  4| 11| asd|  0|  0|1000|10000000|150|12000000|
|1989|  1| 01|null|  1|  4|  30|  300000|500|  100000|
|2001|  1| 02|null|  1|  0| 100|  200000|100|  200000|
|2002|  2| 05|null|  2|  3|  55| 9000000|100| 8000000|
|2007|  1| 02|null|  3|  0| 789|10000000|600|-9000000|
+----+---+---+----+---+---+----+--------+---+--------+



In [3]:
df = df \
    .withColumnRenamed('_c0', 'DEADLINE_YEAR') \
    .withColumnRenamed('_c1', 'DEADLINE_QUARTER') \
    .withColumnRenamed('_c2', 'DEADLINE_MONTH') \
    .withColumnRenamed('_c3', 'KEY_POINT_MGT_IDX') \
    .withColumnRenamed('_c4', 'MGT_BRANCH_CD') \
    .withColumnRenamed('_c5', 'ST_MGT_AREA_CD') \
    .withColumnRenamed('_c6', 'GOAL_HEAD_CNT') \
    .withColumnRenamed('_c7', 'GOAL_AMT') \
    .withColumnRenamed('_c8', 'ACTUAL_HEAD_CNT') \
    .withColumnRenamed('_c9', 'ACTUAL_AMT')

df.show(truncate=False)

+-------------+----------------+--------------+-----------------+-------------+--------------+-------------+--------+---------------+----------+
|DEADLINE_YEAR|DEADLINE_QUARTER|DEADLINE_MONTH|KEY_POINT_MGT_IDX|MGT_BRANCH_CD|ST_MGT_AREA_CD|GOAL_HEAD_CNT|GOAL_AMT|ACTUAL_HEAD_CNT|ACTUAL_AMT|
+-------------+----------------+--------------+-----------------+-------------+--------------+-------------+--------+---------------+----------+
|2019         |1               |02            |null             |0            |0             |500          |1000000 |250            |900000    |
|2010         |4               |12            |asd              |1            |1             |100          |2000000 |450            |1900000   |
|2015         |4               |12            |null             |1            |0             |300          |3000000 |350            |3900000   |
|1994         |2               |04            |null             |2            |0             |50           |500000  |250          

## Constraint Verification

- does df have a size of at least 3?
- does the ACTUAL_HEAD_CNT column have a minimum value of 0?
- is the MGT_BRANCH_CD column complete?
- is the () column unique? => isUnique()
- are the values of DEADLINE_MONTH column contained in "foo", "bar", and "baz"?
- are the values in ACTUAL_HEAD_CNT colum non-negative?

In [4]:
import textwrap
column_name =['DEADLINE_YEAR', 'DEADLINE_QUARTER', 'DEADLINE_MONTH', 'KEY_POINT_MGT_IDX', 'MGT_BRANCH_CD', 'ST_MGT_AREA_CD', 'GOAL_HEAD_CNT', 'GOAL_AMT', 'ACTUAL_HEAD_CNT', 'ACTUAL_AMT']
required = ['N', 'N', 'N', 'N', 'N', 'N', None, None, None, None]

null_lst = []
for i, k in zip(range(len(column_name)),range(len(required))):
    if required[k] == 'N':
        null_lst.append(column_name[i])

        null_check_lst = ""


In [5]:
null_lst

['DEADLINE_YEAR',
 'DEADLINE_QUARTER',
 'DEADLINE_MONTH',
 'KEY_POINT_MGT_IDX',
 'MGT_BRANCH_CD',
 'ST_MGT_AREA_CD']

In [22]:
negativeCheck = ["GOAL_HEAD_CNT","GOAL_AMT","ACTUAL_HEAD_CNT","ACTUAL_AMT"]
negativeCheck

['GOAL_HEAD_CNT', 'GOAL_AMT', 'ACTUAL_HEAD_CNT', 'ACTUAL_AMT']

In [27]:
len(null_lst)

6

In [26]:
a = null_lst + negativeCheck
print(a)

['DEADLINE_YEAR', 'DEADLINE_QUARTER', 'DEADLINE_MONTH', 'KEY_POINT_MGT_IDX', 'MGT_BRANCH_CD', 'ST_MGT_AREA_CD', 'GOAL_HEAD_CNT', 'GOAL_AMT', 'ACTUAL_HEAD_CNT', 'ACTUAL_AMT']


In [34]:
for i, v in enumerate(a):
    if i >= len(null_lst):
        print(i, v)
    

6 GOAL_HEAD_CNT
7 GOAL_AMT
8 ACTUAL_HEAD_CNT
9 ACTUAL_AMT


In [42]:
from pydeequ.checks import *
from pydeequ.verification import *
from pprint import pprint
# null_lst , negativeCheck

check = Check(spark, CheckLevel.Error, "Integrity checks")
checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.isComplete("KEY_POINT_MGT_IDX"))  \
    .run()

checkResult.onData(df).addCheck(check.isNonNegative("DEADLINE_YEAR")).run()

checksuccessResult_df = VerificationResult.successMetricsAsDataFrame(spark, checkResult, pandas=True)


print(f"Verification Run Status: {checkResult.status}")
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult, pandas=True)
checkResult_df


AttributeError: 'VerificationRunBuilder' object has no attribute 'onData'

In [46]:
from pydeequ.checks import *
from pydeequ.verification import *
from pprint import pprint

check = Check(spark, CheckLevel.Error, "Integrity checks")
checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.isComplete("KEY_POINT_MGT_IDX")  \
        .isNonNegative("DEADLINE_YEAR"))  \
    .run()

checksuccessResult_df = VerificationResult.successMetricsAsDataFrame(spark, checkResult, pandas=True)


print(f"Verification Run Status: {checkResult.status}")
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult, pandas=True)
checkResult_df


Verification Run Status: Error


,check,check_level,check_status,constraint,constraint_status,constraint_message
0,Integrity checks,Error,Error,CompletenessConstraint(Completeness(KEY_POINT_...,Failure,Value: 0.2222222222222222 does not meet the co...
1,Integrity checks,Error,Error,ComplianceConstraint(Compliance(DEADLINE_YEAR ...,Success,


In [19]:
checksuccessResult_df

,entity,instance,name,value
0,Column,KEY_POINT_MGT_IDX,Completeness,0.222222
1,Column,DEADLINE_YEAR is non-negative,Compliance,1.000000


In [50]:
negativeCheck

['GOAL_HEAD_CNT', 'GOAL_AMT', 'ACTUAL_HEAD_CNT', 'ACTUAL_AMT']

In [51]:
from pydeequ.checks import *
from pydeequ.verification import *
from pprint import pprint

check = Check(spark, CheckLevel.Error, "Integrity checks")
for i in null_lst:
    print("null_lst : ", i)
    checkResult = VerificationSuite(spark) \
        .onData(df) \
        .addCheck(
            check.isComplete(i))  \
        .run()
    
for i in negativeCheck:
    print("neagtive : ", i)
    checkResult = VerificationSuite(spark) \
        .onData(df) \
        .addCheck(
            check.isNonNegative(i))  \
        .run()

checksuccessResult_df = VerificationResult.successMetricsAsDataFrame(spark, checkResult, pandas=True)


print(f"Verification Run Status: {checkResult.status}")
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult, pandas=True)
checksuccessResult_df



null_lst :  DEADLINE_YEAR
null_lst :  DEADLINE_QUARTER
null_lst :  DEADLINE_MONTH
null_lst :  KEY_POINT_MGT_IDX
null_lst :  MGT_BRANCH_CD
null_lst :  ST_MGT_AREA_CD
neagtive :  GOAL_HEAD_CNT
neagtive :  GOAL_AMT
neagtive :  ACTUAL_HEAD_CNT
neagtive :  ACTUAL_AMT
Verification Run Status: Error


,entity,instance,name,value
0,Column,ACTUAL_AMT is non-negative,Compliance,0.777778
1,Column,DEADLINE_QUARTER,Completeness,1.000000
2,Column,DEADLINE_MONTH,Completeness,1.000000
3,Column,ST_MGT_AREA_CD,Completeness,1.000000
4,Column,GOAL_HEAD_CNT is non-negative,Compliance,1.000000
5,Column,GOAL_AMT is non-negative,Compliance,1.000000
6,Column,ACTUAL_HEAD_CNT is non-negative,Compliance,1.000000
7,Column,MGT_BRANCH_CD,Completeness,1.000000
8,Column,DEADLINE_YEAR,Completeness,1.000000
9,Column,KEY_POINT_MGT_IDX,Completeness,0.222222


In [54]:
checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult, pandas=True)

checkResult_df['constraint_message']

0                                                     
1                                                     
2                                                     
3    Value: 0.2222222222222222 does not meet the co...
4                                                     
5                                                     
6                                                     
7                                                     
8                                                     
9    Value: 0.7777777777777778 does not meet the co...
Name: constraint_message, dtype: object

In [44]:
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Error, "Integrity checks")

checkResult = VerificationSuite(spark) \
    .onData(df) \
    .addCheck(
        check.hasSize(lambda x: x >= 1) \
        .hasMin("GOAL_HEAD_CNT", lambda x: x == 30) \
        .isComplete("KEY_POINT_MGT_IDX")  \
        .isContainedIn("DEADLINE_QUARTER", ["1", "2", "4"]) \
        .isNonNegative("ACTUAL_AMT")) \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

+----------------+-----------+------------+--------------------+-----------------+--------------------+
|           check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+----------------+-----------+------------+--------------------+-----------------+--------------------+
|Integrity checks|      Error|       Error|SizeConstraint(Si...|          Success|                    |
|Integrity checks|      Error|       Error|MinimumConstraint...|          Failure|Expected type of ...|
|Integrity checks|      Error|       Error|CompletenessConst...|          Failure|Value: 0.0 does n...|
|Integrity checks|      Error|       Error|ComplianceConstra...|          Success|                    |
|Integrity checks|      Error|       Error|ComplianceConstra...|          Failure|Value: 0.77777777...|
+----------------+-----------+------------+--------------------+-----------------+--------------------+



In [55]:
if checkResult.status == "Success": 
    print('The data passed the test, everything is fine!')

else:
    print('We found errors in the data, the following constraints were not satisfied:')
    
    for check_json in checkResult.checkResults:
        if check_json['constraint_status'] != "Success": 
            print(f"\t{check_json['constraint']} failed because: {check_json['constraint_message']}")

We found errors in the data, the following constraints were not satisfied:
	CompletenessConstraint(Completeness(KEY_POINT_MGT_IDX,None)) failed because: Value: 0.2222222222222222 does not meet the constraint requirement!
	ComplianceConstraint(Compliance(ACTUAL_AMT is non-negative,COALESCE(CAST(ACTUAL_AMT AS DECIMAL(20,10)), 0.0) >= 0,None)) failed because: Value: 0.7777777777777778 does not meet the constraint requirement!


In [18]:
from pydeequ.checks import *
from pydeequ.verification import *

check = Check(spark, CheckLevel.Error, "Integrity checks")

checkResult = VerificationSuite(spark) \
    .onData(df_spark) \
    .addCheck(
        check.hasSize(lambda x: x >= 3) \
        .hasMin("_c8", lambda x: x == 0) \
        .isComplete("_c7")  \
        .isUnique("_c0")  \
        .isContainedIn("_c0", ["김나래", "이시우", "김지아","윤지혁", "이준석", "최진혁","김진석", "양민아", "송지욱"]) \
        .isNonNegative("_c5")) \
    .run()

checkResult_df = VerificationResult.checkResultsAsDataFrame(spark, checkResult)
checkResult_df.show()

+----------------+-----------+------------+--------------------+-----------------+--------------------+
|           check|check_level|check_status|          constraint|constraint_status|  constraint_message|
+----------------+-----------+------------+--------------------+-----------------+--------------------+
|Integrity checks|      Error|       Error|SizeConstraint(Si...|          Success|                    |
|Integrity checks|      Error|       Error|MinimumConstraint...|          Failure|Expected type of ...|
|Integrity checks|      Error|       Error|CompletenessConst...|          Failure|Value: 0.9 does n...|
|Integrity checks|      Error|       Error|UniquenessConstra...|          Failure|Value: 0.7 does n...|
|Integrity checks|      Error|       Error|ComplianceConstra...|          Failure|Value: 0.9 does n...|
|Integrity checks|      Error|       Error|ComplianceConstra...|          Success|                    |
+----------------+-----------+------------+--------------------+

In [20]:
df_new = checkResult_df.select(checkResult_df.constraint_message)

In [21]:
df_new.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|constraint_message                                                                                                                                                                     |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                       |
|Expected type of column _c8 to be one of (org.apache.spark.sql.types.DecimalType$@67323ea4,LongType,IntegerType,DoubleType,ByteType,FloatType,ShortType), but found StringType instead!|
|Value: 0.9 does not meet the constraint requirement!                 

In [14]:
if checkResult.status == "Success": 
    print('The data passed the test, everything is fine!')

else:
    print('We found errors in the data, the following constraints were not satisfied:')
    
    for check_json in checkResult.checkResults:
        if check_json['constraint_status'] != "Success": 
            print(f"\t{check_json['constraint']} failed because: {check_json['constraint_message']}")

We found errors in the data, the following constraints were not satisfied:
	CompletenessConstraint(Completeness(KEY_POINT_MGT_IDX,None)) failed because: Value: 0.0 does not meet the constraint requirement!
	ComplianceConstraint(Compliance(ACTUAL_AMT is non-negative,COALESCE(CAST(ACTUAL_AMT AS DECIMAL(20,10)), 0.0) >= 0,None)) failed because: Value: 0.7777777777777778 does not meet the constraint requirement!


## Analyzers

In [27]:
from pydeequ.analyzers import *


analysisResult = AnalysisRunner(spark) \
                    .onData(df_spark) \
                    .addAnalyzer(Size()) \
                    .addAnalyzer(Completeness("_c0")) \
                    .addAnalyzer(Completeness("_c1")) \
                    .addAnalyzer(Completeness("_c8")) \
                    .run()
analysisResult_df = AnalyzerContext.successMetricsAsDataFrame(spark, analysisResult)
analysisResult_df.show()

+-------+--------+------------+-----+
| entity|instance|        name|value|
+-------+--------+------------+-----+
|Dataset|       *|        Size| 10.0|
| Column|     _c0|Completeness|  1.0|
| Column|     _c1|Completeness|  1.0|
| Column|     _c8|Completeness|  1.0|
+-------+--------+------------+-----+



## Profile

In [31]:
from pydeequ.profiles import *

result = ColumnProfilerRunner(spark) \
    .onData(df_spark) \
    .run()

for col, profile in result.profiles.items():
    print(profile)

StandardProfiles for column: _c3: {
    "completeness": 1.0,
    "approximateNumDistinctValues": 10,
    "dataType": "String",
    "isDataTypeInferred": false,
    "typeCounts": {
        "Boolean": 0,
        "Fractional": 0,
        "Integral": 0,
        "Unknown": 0,
        "String": 10
    },
    "histogram": [
        [
            "\uae40\uc6a9\ud0dd",
            1,
            0.1
        ],
        [
            "\ub958\ud0dc\uc6b1",
            1,
            0.1
        ],
        [
            "\uacfd\uc7ac\ud601",
            1,
            0.1
        ],
        [
            "\uc815\uc724\ud658",
            1,
            0.1
        ],
        [
            "\uae40\uc778\uaddc",
            1,
            0.1
        ],
        [
            "\uc774\uc11c\uc6a9",
            1,
            0.1
        ],
        [
            "\uae40\ub3d9\ud601",
            1,
            0.1
        ],
        [
            "nm_sponsor",
            1,
            0.1
        ],
 